This first part of the task I will be evaluating my preprocessing. Before starting with the real Sentiment Analysis. 

In [1]:
import json
import nltk
import random
import spacy

# This is the Norwegian language package with all extentions to remove redundant
# filler words from the data:
from spacy.lang.nb import Norwegian 
from spacy.lang.nb.stop_words import STOP_WORDS

data_processing extracts the data from the json file provided in a structured way: ["sentance", "sentiment"]

In [3]:
def data_processing(file_name) -> list:
    message_data = []
    with open(f'3class/{file_name}', "r") as training_data:
        train_array = json.load(training_data)

        #Train_array is a array with dictionaries, each having three elements each. 
        for category in train_array:
            #category is the index of the list with the three elements: sent_id, text, label
            message_data.append([category["text"], category["label"]]) # This appends [text, label] to the array.

        training_data.close() 
            
    return message_data

In [4]:
train = data_processing("train.json")
# test = data_processing("test.json")

# print("Training data: \n", train, "\n", "Testing data: \n", test, "\n")

Training data: 
 [['Philips 190G6', 'Neutral'], ['Med integrerte høyttalere som på ingen måte er diskret plassert , og med en stor subwoofer inkludert , da snakker vi om en gutteskjerm .', 'Neutral'], ['Eller bedrar skinnet ?', 'Negative'], ['De fleste skjermer har et diskret design , med smale rammer og slank fot .', 'Neutral'], ['Men 190G6 fra Philips er en helt annen historie .', 'Neutral'], ['Den har et utseende som krever oppmerksomhet , med glinsende svart ramme , glansbelegg på skjermflaten og store sølvfargede sidepaneler med fire innfelte høyttalere med svart deksel .', 'Neutral'], ['LES OGSÅ :', 'Neutral'], ['Foten har en stor og blank søyle , og det er store knapper og blå lys .', 'Neutral'], ['Baksiden er sort , blank og skinnende , med et deksel som skjuler kontakter og kabler .', 'Positive'], ['De fire høyttalerbrønnene stikker tydelig ut - her er det ikke snakk om å gjemme noe .', 'Neutral'], ['Likegyldig er det uansett vanskelig å være .', 'Neutral'], ['God betjening', 

I heard from my seminar leader that there is a python extention named "spaCy" which is very helpful when it comes to preprocessing the data. And so I found the .lemma_  attribute of spaCy which could imporve the performance of the algorithms even more by lemmatizing the words in the data, in order to make them more understandable for processing. So I ran the train and test data through a spaCy lemmatizer and it was very effective.

Later when working with this project I struggled with removing the non-alphabetic characters from the dataset and more so to do that in an efficient way. Without needing to access the data unnecessary many times. And once again my seminar leader gave me a good advice to performe both lemmatizing and stopwords removal in one function. And so I did.

In [9]:
def the_lemmaSanitizer(data):
    lemmatized_data = []
    lemmatizer = spacy.load("nb_core_news_sm") # Norwegian lemmatization model. 

    for chunk in data: # chunk is a index in data that both contains the sentance and it's sentiment. Not edited. 
        
        that_text_part_of_the_chunk, that_sentiment_part_of_the_chunk = chunk[0], chunk[1]
        
        # pre_lemmed is a sentance that is put in the spacy format, I do this in order
        # to later be able to use nlp's attribute to check for non-alphabetic characters.
        pre_lemmed = lemmatizer(that_text_part_of_the_chunk) 

        lemmed = []
        for root_word in pre_lemmed:
            # I will comment on this if statement later.
            if (not root_word.is_punct
                    and not root_word.is_currency 
                    and not root_word.is_digit
                    and not root_word.is_space
                    and not root_word.is_stop
                    and not root_word.like_num):
                lemmed.append(root_word.lemma_) # .lemma_ finds the lemma of "root_word"
        
        lemmatized_data.append([lemmed, that_sentiment_part_of_the_chunk]) 

    return lemmatized_data

I was considering wether using these if statements in the for loop was a good idea or not. It was either this or using (string).is_alpha function, but the way I choose to implement this also removes stopwords and in general is more flexible to potentially edit later on.   

Thought: Can I be overfitting the data with filtering out all the stopwords? 

In [6]:
train = the_lemmaSanitizer(train)
#test = the_lemmaSanitizer(test)

print("Training data: ", train)
#print("Testing data: ", test)

Training data:  []


NOTE TO MYSELF:
You need to change the "sentance_sanitized_tokening" function to only remove stopwords.
AND not least to set that function before lemmaSanitizer to remove stopwords from data before finding the lemma of it to spare me some time. 

And here starts the tokanization of the lemmatized words using spaCy:

In [4]:
def sentance_sanitized_tokening(data: dict) -> list:
    tokenized_list = []
    dummyWords = nltk.corpus.stopwords.words("norwegian")
    

    # data_part = ["text", "label"]
    for sentance, label in data:
        assert type(sentance) == str, "The sentance variable is not an string!!"

        tokened_sentance = nltk.word_tokenize(sentance)
        
        # Each sentance has it's own filtered_tokens list that only appends the words that is checked to not be stopwords.. 
        filtered_tokens = []
        
        for word in tokened_sentance:
            assert type(word)==str, "Words inside the dictionary passed to tokenize(data) is not of type string!!"
            
            # I considered using remove() function but that would have a much higherruntime which I prefere to avoid. 
            if word.lower() not in dummyWords and word.lower() not in symbols:
                filtered_tokens.append(word.lower())
            else:
                pass
        
        # When all the words from the sentance that were not stopwords was added then append them to the output (tokenized_list)
        if len(filtered_tokens)!=0:
            tokenized_list.append([filtered_tokens, label])
        
    return tokenized_list

When runing the "traing" and "test" variables through this function we get a more clear data_sample with reduced amount of redundant factors. Which helps both the accuracy of the results and the efficiency of the algorithm. 

This is done by removing all the redundant stopwords like: "like" and "and" or "or". It also removes symbols that does not affect our data in any way! So I mean, why not?  

In [5]:
train = sentance_sanitized_tokening(train)
test = sentance_sanitized_tokening(test)

print("Training data: \n", train, "\n", "Testing data: \n", test, "\n")

Training data: 
 [[['philips', '190g6'], 'Neutral'], [['integrerte', 'høyttalere', 'måte', 'diskret', 'plassert', 'stor', 'subwoofer', 'inkludert', 'snakker', 'gutteskjerm'], 'Neutral'], [['bedrar', 'skinnet'], 'Negative'], [['fleste', 'skjermer', 'diskret', 'design', 'smale', 'rammer', 'slank', 'fot'], 'Neutral'], [['190g6', 'philips', 'helt', 'annen', 'historie'], 'Neutral'], [['utseende', 'krever', 'oppmerksomhet', 'glinsende', 'svart', 'ramme', 'glansbelegg', 'skjermflaten', 'store', 'sølvfargede', 'sidepaneler', 'fire', 'innfelte', 'høyttalere', 'svart', 'deksel'], 'Neutral'], [['les'], 'Neutral'], [['foten', 'stor', 'blank', 'søyle', 'store', 'knapper', 'blå', 'lys'], 'Neutral'], [['baksiden', 'sort', 'blank', 'skinnende', 'deksel', 'skjuler', 'kontakter', 'kabler'], 'Positive'], [['fire', 'høyttalerbrønnene', 'stikker', 'tydelig', '-', 'snakk', 'gjemme'], 'Neutral'], [['likegyldig', 'uansett', 'vanskelig'], 'Neutral'], [['god', 'betjening'], 'Positive'], [['midten', 'finner', 'v